# In Search Of Happiness: What If Health?

Let's continue our happiness challenge! Previously, we found out the happiest places on the planet. You can read about this and much more [here](https://nbviewer.jupyter.org/github/chupstee/data.sugar/blob/master/00002_world_happiness/world_happiness.map.ipynb).  
And now we would like to know whether the level of happiness is associated with health.  
We'll take [the same report from Kaggle](https://www.kaggle.com/unsdsn/world-happiness?select=2019.csv), which ranks 156 countries by their level of happiness on a 10-point scale.


## The World Happiness Report

Recall quoting Kaggle:

The World Happiness Report is a landmark survey of the state of global happiness. The first report was published in 2012, the second in 2013, the third in 2015, and the fourth in the 2016 Update. The World Happiness 2017, which ranks 155 countries by their happiness levels, was released at the United Nations at an event celebrating International Day of Happiness on March 20th. The report continues to gain global recognition as governments, organizations and civil society increasingly use happiness indicators to inform their policy-making decisions. Leading experts across fields – economics, psychology, survey analysis, national statistics, health, public policy and more – describe how measurements of well-being can be used effectively to assess the progress of nations. The reports review the state of happiness in the world today and show how the new science of happiness explains personal and national variations in happiness.

You can read more [here](https://www.kaggle.com/unsdsn/world-happiness).

We are most interested in the following columns:

- `Country or region` - country name
- `Overall rank` - country's place in the rating
- `Score` - happiness score


## Introduction

So, what is happiness? Or, better yet, are there any patterns in people's happiness?

[The World Factbook](https://www.cia.gov/library/publications/the-world-factbook) by CIA provides information on the history, people and society, government, economy, energy, geography, communications, transportation, military, and transnational issues for 267 world entities.  
What is interesting for us in this project is to compare the level of happiness of the country and such health indicators as:

- `Obesity` - adult prevalence rate gives the percent of a country's population considered to be obese.
- `Maternal mortality rate` (`MMR`) - the annual number of female deaths per 100,000 live births from any cause related to or aggravated by pregnancy or its management (excluding accidental or incidental causes).
- `Infant mortality rate` - the number of deaths of infants under one year old in a given year per 1,000 live births in the same year. This rate is often used as an indicator of the level of health in a country.
- `Total fertility rate` (`TFR`) - the average number of children that would be born per woman if all women lived to the end of their childbearing years and bore children according to a given fertility rate at each age. TFR is a more direct measure of the level of fertility than the crude birth rate, since it refers to births per woman.
- `HIV/AIDS - adult prevalence rate` - an estimate of the percentage of adults (aged 15-49) living with HIV/AIDS. The adult prevalence rate is calculated by dividing the estimated number of adults living with HIV/AIDS at yearend by the total adult population at yearend.
- `HIV/AIDS - deaths` - the number of adults and children who died of AIDS during a given calendar year.

We will compare the happiness scores with the CIA rates for 2016, as this is the year when the data is presented in the most complete way.  

In the search for an answer to the question ***what*** *is happiness?*, let's try to establish *the relationship between happiness and health indicators*.


## Reading The Data

In [ ]:
# Import libs
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Happiness Reports

As we mentioned above, we'll read the happiness report, published in 2016.  
We'll also rename the columns according to the snake_case format.

In [ ]:
# Set the width to display
pd.set_option('display.width', 120)
# Increase the number of rows to display
pd.set_option('display.max_rows', 60) 

# Get the data
happiness = pd.read_csv('../input/world-happiness/2016.csv')

# Column map to rename
cols_dict = {'Country':'country',
             'Country or region':'country',
             'Region':'region',
             'Happiness Rank':'rank',
             'Happiness.Rank':'rank',
             'Overall rank':'rank',
             'Happiness Score':'score',
             'Happiness.Score':'score',
             'Score':'score',
             'Economy (GDP per Capita)':'gdp_per_capita',
             'Economy..GDP.per.Capita.':'gdp_per_capita',
             'GDP per capita':'gdp_per_capita',
             'Family':'family',
             'Freedom':'freedom',
             'Freedom to make life choices':'freedom',
             'Generosity':'generosity',
             'Health (Life Expectancy)':'life_expectancy',
             'Health..Life.Expectancy.':'life_expectancy',
             'Healthy life expectancy':'life_expectancy',
             'Perceptions of corruption':'trust_corruption',
             'Trust (Government Corruption)':'trust_corruption',
             'Trust..Government.Corruption.':'trust_corruption',
             'Social support':'social_support',
             'Dystopia Residual':'dystopia_residual',
             'Dystopia.Residual':'dystopia_residual',
             'Standard Error':'standard_error',
             'Upper Confidence Interval':'whisker_high',
             'Whisker.high':'whisker_high',
             'Lower Confidence Interval':'whisker_low',
             'Whisker.low':'whisker_low'
            }

# Rename the columns
happiness.rename(columns=cols_dict, inplace=True)

print(happiness.columns) # check the new column names
happiness.head() # check the values

In [ ]:
happiness.info()

We see 157 countries in the report of 2016. There are no missing values for the `country`, `rank`, `score` columns.

Let's check for duplicates.

In [ ]:
# Duplicated
print('Duplicated: {}'.format(happiness.duplicated(subset='country').sum()))

It's OK. Let's get the CIA data.

### CIA Reports

We have downloaded The World Factbook archive for different years and saved the data that was collected in 2016.

In [ ]:
cia_files = glob('/kaggle/input/the-world-factbook-by-cia/cia.health.*.201?.txt')
cia = pd.DataFrame()

for file in cia_files:
    c = pd.read_csv(file,
                    engine='python', sep=r'\s{3,}', header=None, names=['country_cia', file.split('.')[2]],
                    squeeze=False, skiprows=0, index_col=[0],
                    thousands=',', dtype={file.split('.')[2]:'float64'}
                   ) # read the file
    if cia.size == 0:
        cia = cia.append(c)
        print('Initialize: {}'.format(file.split('.')[2], cia.shape[0])) # for the first file
    else:
        cia = cia.merge(c, on='country_cia', how='outer')
        print('Merge {}: {}'.format(file.split('.')[2], cia.shape[0]))

cia.reset_index()

cia.info()
cia

## Preparing The Data Sets

Now we should combine the `happiness` and `cia` datasets.

First, we need to check the columns that will be used for merging.
Country names may differ in data sets, for instance, `eSwatini` and `Swaziland`, `Trinidad and Tobago` and `Trinidad & Tobago`. In this case, the rows will not match.

Before, we store the `country_cia` column of the `cia` in a new column `country`.

In [ ]:
cia['country'] = cia['country_cia']

Let's compare the `country` columns of the `happiness` data set and the `cia` data set.
To do this, we'll combine two data sets using `outer` join.

In [ ]:
happiness_cia = happiness.merge(cia, on='country', how='outer')[['country', 'score', 'infant_mortality']]

pd.set_option('display.max_rows', 100) # increase the number of rows to display
happiness_cia[happiness_cia.isnull().any(axis=1)].sort_values(by=['score', 'country']) # the countries don't match

To provide the same country names:

- Create a dictionary mapping all names to the values in the `happiness` dataset since we explore the happiness data.
- Rename the countries in the `cia` dataset by replacing the values according to the map dictionary.

In [ ]:
# Countries map to rename
country_to_rename = {'Cote d\'Ivoire':'Ivory Coast',
                     'Congo, Republic of the':'Congo (Brazzaville)',
                     'Congo, Democratic Republic of the':'Congo (Kinshasa)',
                     'Burma':'Myanmar',
                     'Korea, South':'South Korea',
                     'Czechia':'Czech Republic'
                    }
# Rename the countries
cia['country'].replace(country_to_rename, inplace=True)

cia.sample(5, random_state=5) # check the values randomly

We can now use the `country` column to merge the two datasets.

In [ ]:
happiness_cia = happiness.merge(cia, on='country', how='left').copy()

happiness_cia.info()

Let's build a correlation matrix by columns.

In [ ]:
# Select the columns of interest
cols_corr = ['country', 'region', 'score',
             'obesity', 'tfr',
             'mmr', 'infant_mortality',
             'hiv_aids', 'hiv_aids_death'
            ]
happiness_cia = happiness_cia[cols_corr]
happiness_cia

In [ ]:
# Get correlation matrix
happiness_cia_corr = happiness_cia.corr()
happiness_cia_corr

It seems, there are explicitely relationships between some parameters. We'll look at them further.  
Let's continue with the plots.

## Correlation Map

We'll plot the correlation matrix below, but first, we'll prepare the data to make the plot easier to read.

In [ ]:
# A triangular mask to avoid repeated values
happiness_cia_corr = happiness_cia_corr.iloc[1:, :-1]
mask = np.triu(np.ones_like(happiness_cia_corr), k=1)

# Readable names for the plot
cols_dict = {'score':'Happiness',
             'hiv_aids':'HIV/AIDS',
             'hiv_aids_death':'HIV/AIDS\ndeath',
             'infant_mortality':'Infant\nmortality',
             'mmr':'Maternal\nmortality',
             'tfr':'Total\nfertility',
             'obesity':'Obesity'
            }
# Rename columns in the correlation matrix
happiness_cia_corr.rename(columns=cols_dict, index=cols_dict, inplace=True)

In [ ]:
%matplotlib inline

# Turn on svg rendering
%config InlineBackend.figure_format = 'svg'

# Color palette for the blog
snark_palette = ['#e0675a', # red
                 '#5ca0af', # green
                 '#edde7e', # yellow
                 '#211c47' # dark blue
                ]

In [ ]:
# Color palette for the data
palette = [snark_palette[0], # red
           'lightgrey',
           snark_palette[1] # green
          ]

# Inscriptions
title = """Relationship Between Health Indicators And The Happiness score"""
description = """
Сorrelation of health indicators with the happiness score by country based on 2016 data.
Data: Gallup World Poll - www.kaggle.com/unsdsn/world-happiness & CIA - www.cia.gov/library/publications/the-world-factbook | Author: @data.sugar
"""

# Plot size
figsize = (6,4)

# Set the figure
sns.set(context='paper', style='ticks', palette=palette,
        rc={'xtick.bottom':False, 'ytick.left':False, 
            'axes.spines.left': False, 'axes.spines.bottom': False,
            'axes.spines.right': False, 'axes.spines.top': False
           }
       )

# Create the plot
fig, ax = plt.subplots(1, 1, figsize=figsize, facecolor='w')
sns.heatmap(happiness_cia_corr, mask=mask, cmap=palette,
            vmin=-1, vmax=1, center=0,
            square=False, linewidths=.5, annot=True, fmt='.2g',
            cbar_kws={'shrink': 1, 'ticks':[], 'label':'-1 negative <- correlation -> positive +1'},
            ax=ax)

# Set some aesthetic params for the plot
ax.set_title(title, fontdict={'fontsize': 16}, loc='center', pad=10, c=snark_palette[-1]) # set a title of the plot
ax.annotate(description, xy=(20, -4), size=6, xycoords='figure points', c=snark_palette[-1])
ax.tick_params(axis='both', colors=snark_palette[-1]) # set x/y ticks

# Save and plot
plt.savefig('/kaggle/working/plot.happiness.health.png', dpi=150, bbox_inches='tight')
plt.show()

We expected some correlation between the HIV/AIDS rates and the happiness score, but there is no such correlation.  
Indeed, the higher the level of obesity, the higher the happiness index, but this relationship is not very strict.  
Let's plot these correlations.

## Obesity And Happiness

In [ ]:
# Inscriptions
title = """The Relationship Between Obesity And Happiness"""
description = """
Correlation of the obesity rate with the happiness score by country based on 2016 data.
Data: Gallup World Poll - www.kaggle.com/unsdsn/world-happiness & CIA - www.cia.gov/library/publications/the-world-factbook | Author: @data.sugar
"""

# Plot size
figsize = (6,4)

# Set the figure
sns.set(context='paper', style='ticks', palette=snark_palette,
        rc={'xtick.major.size': 4, 'ytick.major.size':4,
            'axes.spines.left': False, 'axes.spines.bottom': False,
            'axes.spines.right': False, 'axes.spines.top': False
           }
       )

# Create the plot
fig = plt.figure(figsize=figsize, facecolor='w')
g = sns.jointplot(x='score', y='obesity', data=happiness_cia,
                  kind='reg', truncate=False, dropna=True,
                  xlim=(2, 8), ylim=(0, 40),
                  marginal_kws=dict(hist=True, bins=10),
                  color=snark_palette[1]
                 )

# Set some aesthetic params for the plot
g.ax_marg_x.set_title(title, fontdict={'fontsize': 16}, loc='center', pad=10, c=snark_palette[-1]) # set a title of the plot
g.ax_marg_x.annotate(description, xy=(0.015, -0.01), size=6, xycoords='figure fraction', c=snark_palette[-1])
g.ax_joint.set_xlabel('Happiness score', horizontalalignment='center', size='x-large', c=snark_palette[-1]) # set label of x axis
g.ax_joint.set_ylabel('Obesity rate', horizontalalignment='center', size='x-large', c=snark_palette[-1]) # set label of x axis
g.ax_joint.tick_params(axis='both', labelsize='large', colors=snark_palette[-1]) # set x/y ticks
g.ax_joint.spines['bottom'].set_color(snark_palette[-1]) # color x axis
g.ax_joint.spines['left'].set_color(snark_palette[-1]) # color y axis
g.ax_marg_x.tick_params(axis='x', bottom=False) # disable x margin ticks
g.ax_marg_x.spines['bottom'].set_color(snark_palette[1])
g.ax_marg_y.tick_params(axis='y', left=False) # disable y margin ticks
g.ax_marg_y.spines['left'].set_color(snark_palette[1])

# Save and plot
plt.savefig('/kaggle/working/plot.happiness.health.obesity.png', dpi=150, bbox_inches='tight')
plt.show()

## Total Fertility Rate And Happiness Score

In [ ]:
# Inscriptions
title = """The Relationship Between Total Fertility And Happiness"""
description = """
Correlation of the total fertility rate with the happiness score by country based on 2016 data.
Data: Gallup World Poll - www.kaggle.com/unsdsn/world-happiness & CIA - www.cia.gov/library/publications/the-world-factbook | Author: @data.sugar
"""

# Plot size
figsize = (6,4)

# Set the figure
sns.set(context='paper', style='ticks', palette=snark_palette,
        rc={'xtick.major.size': 4, 'ytick.major.size':4,
            'axes.spines.left': False, 'axes.spines.bottom': False,
            'axes.spines.right': False, 'axes.spines.top': False
           }
       )

# Create the plot
fig = plt.figure(figsize=figsize, facecolor='w')
g = sns.jointplot(x='score', y='tfr', data=happiness_cia,
                  kind='reg', truncate=False, dropna=True,
                  xlim=(2, 8), ylim=(0, 7),
                  marginal_kws=dict(hist=True, bins=8),
                  color=snark_palette[0]
                 )

# Set some aesthetic params for the plot
g.ax_marg_x.set_title(title, fontdict={'fontsize': 16}, loc='center', pad=10, c=snark_palette[-1]) # set a title of the plot
g.ax_marg_x.annotate(description, xy=(0.015, -0.01), size=6, xycoords='figure fraction', c=snark_palette[-1])
g.ax_joint.set_xlabel('Happiness score', horizontalalignment='center', size='x-large', c=snark_palette[-1]) # set label of x axis
g.ax_joint.set_ylabel('Total fertility rate', horizontalalignment='center', size='x-large', c=snark_palette[-1]) # set label of x axis
g.ax_joint.tick_params(axis='both', labelsize='large', colors=snark_palette[-1]) # set x/y ticks
g.ax_joint.spines['bottom'].set_color(snark_palette[-1]) # color x axis
g.ax_joint.spines['left'].set_color(snark_palette[-1]) # color y axis
g.ax_marg_x.tick_params(axis='x', bottom=False) # disable x margin ticks
g.ax_marg_x.spines['bottom'].set_color(snark_palette[0])
g.ax_marg_y.tick_params(axis='y', left=False) # disable y margin ticks
g.ax_marg_y.spines['left'].set_color(snark_palette[0])

# Save and plot
plt.savefig('/kaggle/working/plot.happiness.health.tfr.png', dpi=150, bbox_inches='tight')
plt.show()

## Infant Mortality Rate And Happiness Score

In [ ]:
# Inscriptions
title = """The Relationship Between Infant Mortality And Happiness"""
description = """
Correlation of the infant mortality rate with the happiness score by country based on 2016 data.
Data: Gallup World Poll - www.kaggle.com/unsdsn/world-happiness & CIA - www.cia.gov/library/publications/the-world-factbook | Author: @data.sugar
"""

# Plot size
figsize = (6,4)

# Set the figure
sns.set(context='paper', style='ticks', palette=snark_palette,
        rc={'xtick.major.size': 4, 'ytick.major.size':4,
            'axes.spines.left': False, 'axes.spines.bottom': False,
            'axes.spines.right': False, 'axes.spines.top': False
           }
       )

# Create the plot
fig = plt.figure(figsize=figsize, facecolor='w')
g = sns.jointplot(x='score', y='infant_mortality', data=happiness_cia,
                  kind='reg', truncate=False, dropna=True,
                  xlim=(2, 8), ylim=(0, 125),
                  marginal_kws=dict(hist=True, bins=10),
                  color=snark_palette[0]
                 )

# Set some aesthetic params for the plot
g.ax_marg_x.set_title(title, fontdict={'fontsize': 16}, loc='center', pad=10, c=snark_palette[-1]) # set a title of the plot
g.ax_marg_x.annotate(description, xy=(0.015, -0.01), size=6, xycoords='figure fraction', c=snark_palette[-1])
g.ax_joint.set_xlabel('Happiness score', horizontalalignment='center', size='x-large', c=snark_palette[-1]) # set label of x axis
g.ax_joint.set_ylabel('Infant mortality rate', horizontalalignment='center', size='x-large', c=snark_palette[-1]) # set label of x axis
g.ax_joint.tick_params(axis='both', labelsize='large', colors=snark_palette[-1]) # set x/y ticks
g.ax_joint.spines['bottom'].set_color(snark_palette[-1]) # color x axis
g.ax_joint.spines['left'].set_color(snark_palette[-1]) # color y axis
g.ax_marg_x.tick_params(axis='x', bottom=False) # disable x margin ticks
g.ax_marg_x.spines['bottom'].set_color(snark_palette[0])
g.ax_marg_y.tick_params(axis='y', left=False) # disable y margin ticks
g.ax_marg_y.spines['left'].set_color(snark_palette[0])

# Save and plot
plt.savefig('/kaggle/working/plot.happiness.health.infant_mortality.png', dpi=150, bbox_inches='tight')
plt.show()

## Total Fertility Rate And Obesity Rate

In [ ]:
# Inscriptions
title = """The Relationship Between Total Fertility And Obesity"""
description = """
Correlation of the total fertility rate with the obesity rate by country based on 2016 data.
Data: Gallup World Poll - www.kaggle.com/unsdsn/world-happiness & CIA - www.cia.gov/library/publications/the-world-factbook | Author: @data.sugar
"""

# Plot size
figsize = (6,4)

# Set the figure
sns.set(context='paper', style='ticks', palette=snark_palette,
        rc={'xtick.major.size': 4, 'ytick.major.size':4,
            'axes.spines.left': False, 'axes.spines.bottom': False,
            'axes.spines.right': False, 'axes.spines.top': False
           }
       )

# Create the plot
fig = plt.figure(figsize=figsize, facecolor='w')
g = sns.jointplot(x='obesity', y='tfr', data=happiness_cia,
                  kind='reg', truncate=False, dropna=True,
                  xlim=(0, 40), ylim=(0, 7),
                  marginal_kws=dict(hist=True, bins=10),
                  color=snark_palette[0]
                 )

# Set some aesthetic params for the plot
g.ax_marg_x.set_title(title, fontdict={'fontsize': 16}, loc='center', pad=10, c=snark_palette[-1]) # set a title of the plot
g.ax_marg_x.annotate(description, xy=(0.015, -0.01), size=6, xycoords='figure fraction', c=snark_palette[-1])
g.ax_joint.set_xlabel('Obesity rate', horizontalalignment='center', size='x-large', c=snark_palette[-1]) # set label of x axis
g.ax_joint.set_ylabel('Total fertility rate', horizontalalignment='center', size='x-large', c=snark_palette[-1]) # set label of x axis
g.ax_joint.tick_params(axis='both', labelsize='large', colors=snark_palette[-1]) # set x/y ticks
g.ax_joint.spines['bottom'].set_color(snark_palette[-1]) # color x axis
g.ax_joint.spines['left'].set_color(snark_palette[-1]) # color y axis
g.ax_marg_x.tick_params(axis='x', bottom=False) # disable x margin ticks
g.ax_marg_x.spines['bottom'].set_color(snark_palette[0])
g.ax_marg_y.tick_params(axis='y', left=False) # disable y margin ticks
g.ax_marg_y.spines['left'].set_color(snark_palette[0])

# Save and plot
plt.savefig('/kaggle/working/plot.happiness.health.tfr_obesity.png', dpi=150, bbox_inches='tight')
plt.show()

## Conclusions

As we mentioned above, there is no significant correlation between the HIV/AIDS rates and the happiness score.

And these correlations are quite high (above 0.8):

- total fertility rate and infant mortality rate;
- maternal mortality rate and infant mortality rate;
- maternal mortality rate and total fertility rate.

And it seems natural. If the birth rate per woman is higher, the risk of maternal and infant mortality is higher, given the fact that total fertility, maternal mortality, and infant mortality are inversely proportional to the level of happiness.  In other words, the birth rate, maternal and infant mortality are higher in the least happy countries.  

Statistics do not give the root cause: people are happy, and therefore infant mortality is lower, or mortality is lower, and therefore people are happier (because they are generally healthier). Which came first - the chicken or the egg.  

There is a low negative correlation between the total fertility rate and the obesity rate.  
What does it mean? Does this mean that satisfied people don't want to think about procreation?

What about happiness? 
The higher a country is on the happiness stairs, the higher the obesity rate, but the lower the total fertility, as well as maternal and infant mortality.